# Ingestion Experimentation notebook

This notebook largely exists to experiment with different config methods

## Configuration

These are some fields to configure if you wish to configure how the data is injected.

In [1]:
from lib.service.docker.defaults import INSTANCE_2_IMAGE_CONF, INSTANCE_2_CONTAINER_CONF
from lib.service.database.defaults import instance_2_config

# If you mark this as true, the table `nsw_valuer_general.raw_entries`
# will be dropped. If you have space limitations and no desire to debug
# the data than dropping this makes sense. If you wish to debug some values
# then keeping this around may make some sense.
GLOBAL_FLAGS = {
    'drop_raw_nsw_valuer_general_entries': True,
    'reinitialise_container': True,
}

db_service_config = instance_2_config
docker_image_conf = INSTANCE_2_IMAGE_CONF
docker_container_conf = INSTANCE_2_CONTAINER_CONF

## Download Static Files

In [2]:
import logging
from lib.service.io import IoService
from lib.tasks.fetch_static_files import initialise, get_session

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

io_service = IoService.create(None)
async with get_session(io_service) as session:
    environment = await initialise(io_service, session)

land_value_dis = environment.land_value
w_sale_price = environment.sale_price_weekly
a_sale_price = environment.sale_price_annual
gnaf_dis = environment.gnaf

2024-10-04 10:47:11,534 - INFO - Checking Target "abs_main_structures.zip"
2024-10-04 10:47:11,534 - INFO - Checking Target "non_abs_shape.zip"
2024-10-04 10:47:11,534 - INFO - Checking Target "g-naf_aug24_allstates_gda2020_psv_1016.zip"
2024-10-04 10:47:11,535 - INFO - Checking Target "nswvg_lv_01_Oct_2024.zip"
2024-10-04 10:47:11,535 - INFO - Checking Target "nswvg_wps_01_Jan_2024.zip"
2024-10-04 10:47:11,535 - INFO - Checking Target "nswvg_wps_08_Jan_2024.zip"
2024-10-04 10:47:11,535 - INFO - Checking Target "nswvg_wps_15_Jan_2024.zip"
2024-10-04 10:47:11,536 - INFO - Checking Target "nswvg_wps_22_Jan_2024.zip"
2024-10-04 10:47:11,536 - INFO - Checking Target "nswvg_wps_29_Jan_2024.zip"
2024-10-04 10:47:11,536 - INFO - Checking Target "nswvg_wps_05_Feb_2024.zip"
2024-10-04 10:47:11,537 - INFO - Checking Target "nswvg_wps_12_Feb_2024.zip"
2024-10-04 10:47:11,537 - INFO - Checking Target "nswvg_wps_19_Feb_2024.zip"
2024-10-04 10:47:11,537 - INFO - Checking Target "nswvg_wps_26_Feb_202

## Create Docker Container and Database

In [3]:
from lib.pipeline.gnaf.init_schema import init_target_schema
from lib.service.docker import DockerService
from lib.service.database import DatabaseService

docker_service = DockerService.create()

if GLOBAL_FLAGS['reinitialise_container']:
    image = docker_service.create_image(docker_image_conf)
    image.prepare()

    container = docker_service.create_container(image, docker_container_conf)
    container.clean()
    container.prepare(db_service_config)
    container.start()
else:
    print('skipping container initialisation')

db_service = DatabaseService.create(db_service_config, 32)
await db_service.wait_till_running()
await db_service.open()

if GLOBAL_FLAGS['reinitialise_container']:
    await init_target_schema(gnaf_dis.publication, io_service, db_service)
else:
    print('skipping DB initialisation')
    raise Exception()

2024-10-04 10:47:12,422 - WARNING - error connecting in 'pool-1': connection failed: connection to server at "127.0.0.1", port 5433 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
2024-10-04 10:47:12,423 - WARNING - error connecting in 'pool-1': connection failed: connection to server at "127.0.0.1", port 5433 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
2024-10-04 10:47:12,426 - WARNING - error connecting in 'pool-1': connection failed: connection to server at "127.0.0.1", port 5433 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
2024-10-04 10:47:12,432 - WARNING - error connecting in 'pool-1': connection failed: connection to server at "127.0.0.1", port 5433 failed: server closed the connection 

dbname=gnaf_db_2 port=5433 user=postgres host=localhost password=throwAwayPassword2


2024-10-04 10:47:17,440 - INFO - running ./_out_zip/g-naf_aug24_allstates_gda2020_psv_1016/G-NAF/Extras/GNAF_TableCreation_Scripts/create_tables_ansi.sql
/Users/angus/code/jupyter/env/lib/python3.12/site-packages/psycopg_pool/pool_async.py:138: RuntimeWarning: opening the async pool AsyncConnectionPool in the constructor is deprecated and will not be supported anymore in a future release. Please use `await pool.open()`, or use the pool as context manager using: `async with AsyncConnectionPool(...) as pool: `...
  warnings.warn(
2024-10-04 10:47:17,460 - INFO - running ./_out_zip/g-naf_aug24_allstates_gda2020_psv_1016/G-NAF/Extras/GNAF_TableCreation_Scripts/add_fk_constraints.sql
2024-10-04 10:47:17,505 - INFO - running sql/move_gnaf_to_schema.sql


## Init DB Schema

In [4]:
from lib.tasks.update_schema import update_schema, UpdateSchemaConfig

await update_schema(
    UpdateSchemaConfig(apply=True),
    db_service,
    io_service,
)

2024-10-04 10:47:17,525 - INFO - initalising nsw_vg db schema
2024-10-04 10:47:17,537 - INFO - running sql/meta/schema/001_APPLY_init.sql
2024-10-04 10:47:17,550 - INFO - running sql/abs/schema/001_APPLY_init.sql
2024-10-04 10:47:17,554 - INFO - running sql/nsw_lrs/schema/001_APPLY_init.sql
2024-10-04 10:47:17,558 - INFO - running sql/nsw_environment/schema/001_APPLY_init.sql
2024-10-04 10:47:17,564 - INFO - running sql/nsw_property/schema/001_APPLY_init.sql
2024-10-04 10:47:17,576 - INFO - running sql/nsw_vg/schema/001_APPLY_init.sql
2024-10-04 10:47:17,579 - INFO - running sql/nsw_vg/schema/002_APPLY_create_raw.sql
2024-10-04 10:47:17,591 - INFO - running sql/nsw_vg/schema/003_APPLY_shared_tables.sql
2024-10-04 10:47:17,605 - INFO - running sql/nsw_vg/schema/004_APPLY_land_value_tables.sql


## Ingest ABS Data

In [5]:
from lib.tasks.ingest_abs import ingest_all

await ingest_all(db_service)

2024-10-04 10:47:21,011 - INFO - Populated abs_main_structures.state with 10/10 rows.
2024-10-04 10:47:23,834 - INFO - Populated abs_main_structures.gccsa with 35/35 rows.
2024-10-04 10:47:27,588 - INFO - Populated abs_main_structures.sa4 with 108/108 rows.
2024-10-04 10:47:31,993 - INFO - Populated abs_main_structures.sa3 with 359/359 rows.
2024-10-04 10:47:38,422 - INFO - Populated abs_main_structures.sa2 with 2473/2473 rows.
2024-10-04 10:47:53,986 - INFO - Populated abs_main_structures.sa1 with 61845/61845 rows.
2024-10-04 10:48:39,970 - INFO - Populated abs_main_structures.meshblock with 368286/368286 rows.
2024-10-04 10:48:53,038 - INFO - Populated non_abs_main_structures.localities with 15353/15353 rows.
2024-10-04 10:48:57,274 - INFO - Populated non_abs_main_structures.state_electoral_division_2021 with 452/452 rows.
2024-10-04 10:49:01,590 - INFO - Populated non_abs_main_structures.state_electoral_division_2022 with 452/452 rows.
2024-10-04 10:49:06,286 - INFO - Populated non_

## Ingest NSW Valuer General Land Values

In [6]:
from lib.tasks.nsw_vg.ingest_land_values import ingest_land_values, NswVgLandValueIngestionConfig

await ingest_land_values(
    NswVgLandValueIngestionConfig(
        keep_raw=not GLOBAL_FLAGS['drop_raw_nsw_valuer_general_entries'],
    ),
    db_service,
    environment.land_value.latest,
)

2024-10-04 10:49:44,834 - INFO - Step 1: Ingest raw files
/Users/angus/code/jupyter/notebooks/20240907, vg/lib/pipeline/nsw_vg/land_values/ingest.py:63: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_file_path, encoding='utf-8')
/Users/angus/code/jupyter/notebooks/20240907, vg/lib/pipeline/nsw_vg/land_values/ingest.py:63: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_file_path, encoding='utf-8')
/Users/angus/code/jupyter/notebooks/20240907, vg/lib/pipeline/nsw_vg/land_values/ingest.py:63: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_file_path, encoding='utf-8')
/Users/angus/code/jupyter/notebooks/20240907, vg/lib/pipeline/nsw_vg/land_values/ingest.py:63: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  d

## Ingest Gnaf

In [7]:
from lib.tasks.ingest_gnaf import ingest_gnaf
await ingest_gnaf(gnaf_dis.publication, db_service)

2024-10-04 10:56:19,562 - INFO - Populating from Authority_Code_MB_MATCH_CODE_AUT_psv.psv
2024-10-04 10:56:19,562 - INFO - Populating from Authority_Code_STREET_TYPE_AUT_psv.psv
2024-10-04 10:56:19,563 - INFO - Populating from Authority_Code_FLAT_TYPE_AUT_psv.psv
2024-10-04 10:56:19,563 - INFO - Populating from Authority_Code_GEOCODE_RELIABILITY_AUT_psv.psv
2024-10-04 10:56:19,564 - INFO - Populating from Authority_Code_GEOCODE_TYPE_AUT_psv.psv
2024-10-04 10:56:19,564 - INFO - Populating from Authority_Code_ADDRESS_CHANGE_TYPE_AUT_psv.psv
2024-10-04 10:56:19,566 - INFO - Populating from Authority_Code_LOCALITY_CLASS_AUT_psv.psv
2024-10-04 10:56:19,568 - INFO - Populating from Authority_Code_GEOCODED_LEVEL_TYPE_AUT_psv.psv
2024-10-04 10:56:19,570 - INFO - Populating from Authority_Code_LEVEL_TYPE_AUT_psv.psv
2024-10-04 10:56:19,570 - INFO - Populating from Authority_Code_ADDRESS_ALIAS_TYPE_AUT_psv.psv
2024-10-04 10:56:19,570 - INFO - Populating from Authority_Code_STREET_SUFFIX_AUT_psv.

## Done

In [8]:
async with (
    await db_service.async_connect() as c,
    c.cursor() as cursor
):
    for schema in ['nsw_valuer_general', 'gnaf', 'abs_main_structures', 'non_abs_main_structures']:
        # Get the list of all tables
        cursor.execute(f"""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = '{schema}'
        """)
        tables = cursor.fetchall()
    
        # Get row count for each table
        for table in tables:
            await cursor.execute(f'SELECT COUNT(*) FROM {schema}.{table[0]}')
            count = cursor.fetchone()[0]
            print(f"Table {schema}.{table[0]} has {count} rows")

TypeError: object _AsyncGeneratorContextManager can't be used in 'await' expression